# M4L Demucs Worker (One Notebook)

Run all cells once. Drops-only workflow is default. Advanced users can flip `zero_config` in Drive config.


In [ ]:
# Cell 1: Mount Drive and show GPU (warn if CPU)
from google.colab import drive
import subprocess

drive.mount('/content/drive', force_remount=True)
smi = subprocess.getoutput('nvidia-smi')
print(smi)
if 'NVIDIA-SMI' not in smi:
    print('\nWARNING: GPU not detected. Go to Runtime > Change runtime type > Select GPU, then rerun.')


In [ ]:
# Cell 2: Fast bootstrap — wheelhouse + cached site-packages + model cache
import os, sys, subprocess, pathlib, re, json
BASE = pathlib.Path('/content/drive/MyDrive/M4L-Demucs')
SITE = BASE / '.venv' / 'site-packages'
WHEELS = BASE / 'wheelhouse'
PIP_CACHE = BASE / 'pip-cache'
MODEL_CACHE = BASE / 'model-cache'
MARKER = BASE / '.env.json'
for p in [SITE, WHEELS, PIP_CACHE, MODEL_CACHE]: p.mkdir(parents=True, exist_ok=True)

os.environ.setdefault('XDG_CACHE_HOME', str(MODEL_CACHE))
os.environ.setdefault('DEMUCS_CACHE', str(MODEL_CACHE))

if str(SITE) not in sys.path:
    sys.path.insert(0, str(SITE))

def detect_torch_index():
    smi = subprocess.getoutput('nvidia-smi')
    m = re.search(r'CUDA Version:\s*([0-9.]+)', smi)
    if m:
        ver = m.group(1)
        if ver.startswith('12.6'): return 'https://download.pytorch.org/whl/cu126', 'cu126'
        if ver.startswith('12.1'): return 'https://download.pytorch.org/whl/cu121', 'cu121'
        if ver.startswith('11.8'): return 'https://download.pytorch.org/whl/cu118', 'cu118'
    return 'https://download.pytorch.org/whl/cu121', 'cu121'

# Desired versions (bump here when upgrading)
TORCH_VER = '2.4.0'
TORCHAUDIO_VER = '2.4.0'
DEMUCS_VER = '4.0.1'

# Load marker; decide if refresh needed
want = {}
idx_url, flavor = detect_torch_index()
want = { 'torch': f'{TORCH_VER}+{flavor}', 'torchaudio': f'{TORCHAUDIO_VER}+{flavor}', 'demucs': DEMUCS_VER }
cur = {}
if MARKER.exists():
    try: cur = json.loads(MARKER.read_text() or '{}')
    except Exception: cur = {}
need_refresh = (cur != want)

# Helper to run pip
def run(cmd):
    return subprocess.run(cmd, text=True).returncode

# Ensure wheels exist (first time only or if versions changed)
if need_refresh:
    print('Downloading wheels to Drive (one-time)…')
    cmd = ['pip','download','-d',str(WHEELS),'--prefer-binary','--only-binary=:all:','--extra-index-url',idx_url,
           f'torch=={TORCH_VER}+{flavor}', f'torchaudio=={TORCHAUDIO_VER}+{flavor}', f'demucs=={DEMUCS_VER}', 'ffmpeg-python','numpy']
    code = run(cmd)
    if code != 0 and flavor != 'cu118':
        # fallback to cu118
        print('Primary CUDA wheels unavailable. Falling back to cu118…')
        idx_url, flavor = 'https://download.pytorch.org/whl/cu118', 'cu118'
        cmd = ['pip','download','-d',str(WHEELS),'--prefer-binary','--only-binary=:all:','--extra-index-url',idx_url,
               f'torch=={TORCH_VER}+{flavor}', f'torchaudio=={TORCHAUDIO_VER}+{flavor}', f'demucs=={DEMUCS_VER}', 'ffmpeg-python','numpy']
        if run(cmd) != 0:
            raise RuntimeError('Failed to download torch/demucs wheels for cu118/cu121/cu126')
    MARKER.write_text(json.dumps(want))

# Install from wheelhouse into Drive site-packages (fast, offline)
print('Installing from wheelhouse into site-packages…')
run([sys.executable,'-m','pip','install','-q','--no-warn-script-location','--no-index','--find-links',str(WHEELS),
     '--upgrade','--target',str(SITE),'torch','torchaudio','demucs','ffmpeg-python','numpy'])
# Quiet fix for IPython warning
run([sys.executable,'-m','pip','install','-q','--target',str(SITE),'jedi>=0.16'])

import demucs, torch  # noqa
print('Boot OK. demucs', demucs.__version__, '| torch', torch.__version__, 'cuda', torch.cuda.is_available(), '| wheelhouse', WHEELS)
print('site-packages:', SITE)
print('model-cache:', MODEL_CACHE)


In [ ]:
# Cell 3: Config reader (zero_config default true)
import json, pathlib
ROOT = pathlib.Path('/content/drive/MyDrive/M4L-Demucs')
ROOT.mkdir(parents=True, exist_ok=True)
CONFIG = ROOT / 'config.json'
if not CONFIG.exists():
    CONFIG.write_text(json.dumps({"zero_config": True}))
print('Config:', CONFIG.read_text())


In [ ]:
# Cell 4: Watcher loop
import urllib.request, subprocess, sys
watcher_url = 'https://raw.githubusercontent.com/VSTOPIA/Doctor-Sample-Unit-DSU/main/colab_watcher.py'
urllib.request.urlretrieve(watcher_url, ROOT / 'colab_watcher.py')
print('Watcher fetched')
!python /content/drive/MyDrive/M4L-Demucs/colab_watcher.py


In [ ]:
# Optional: one-shot run for a specific file (debug)
# Place a file at /content/drive/MyDrive/M4L-Demucs/jobs/audio/YourSong.wav
# Then uncomment and run below:
# import subprocess, sys, pathlib
# IN_WAV = pathlib.Path('/content/drive/MyDrive/M4L-Demucs/jobs/audio/YourSong.wav')
# OUT = pathlib.Path('/content/drive/MyDrive/M4L-Demucs/out/oneshot')
# OUT.mkdir(parents=True, exist_ok=True)
# cmd = [sys.executable,'-m','demucs.separate','-n','htdemucs_ft','-o',str(OUT),'-j','4','--shifts','4','--two-stems','vocals',str(IN_WAV)]
# print(' '.join(cmd))
# subprocess.run(cmd)


In [ ]:
# Diagnostics: confirm GPU + cached installs
import sys, subprocess, pathlib, json
BASE = pathlib.Path('/content/drive/MyDrive/M4L-Demucs')
SITE = BASE/'.venv'/'site-packages'
WHEELS = BASE/'wheelhouse'
MODELS = BASE/'model-cache'
print('python:', sys.version)
print('nvidia-smi:')
print(subprocess.getoutput('nvidia-smi'))
try:
  import torch, demucs
  print('torch:', torch.__version__, 'cuda avail:', torch.cuda.is_available(), 'torch cuda:', torch.version.cuda)
  print('demucs:', demucs.__version__)
  print('demucs path:', getattr(demucs,'__file__','?'))
  print('torch path:', getattr(torch,'__file__','?'))
except Exception as e:
  print('Import error:', e)
print('site-packages exists:', SITE.exists(), 'wheelhouse exists:', WHEELS.exists(), 'models exists:', MODELS.exists())
